# Analysing binned pseudorapidity

In [17]:
# System imports
import os
import sys
from pprint import pprint as pp
from time import time as tt
import pickle

# External imports
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import multiprocessing as mp
from functools import partial
import numpy.lib.recfunctions as rfn

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
import torch_geometric
from torch_geometric.data import Data

# Limit CPU usage on Jupyter
os.environ['OMP_NUM_THREADS'] = '4'

# Pick up local packages
sys.path.append('..')

# Profiling
# %load_ext heat

# Locals

from prepareTracks import *
from nb_utils import *
from gpu_utils import *
from datasets import get_data_loaders
from trainers import get_trainer
import distributed
import numba

## Make File List

In [31]:
input_dir = '/global/cscratch1/sd/danieltm/ExaTrkX/trackml/train_100_events/'
output_dir = '/global/u2/d/danieltm/ExaTrkX/eta-tracker/data/numba_testing'
n_files = 1
n_workers = 1
config = {'selection': {'pt_min': 0.5,
    'phi_slope_max': 0.0006,
    'z0_max': 150,
    'n_phi_sections': 1,
    'n_eta_sections': 1,
    'eta_range': [-5, 5]}}

In [32]:
all_files = os.listdir(input_dir)
suffix = '-hits.csv'
file_prefixes = sorted(os.path.join(input_dir, f.replace(suffix, ''))
                       for f in all_files if f.endswith(suffix))
file_prefixes = np.array(file_prefixes[:n_files])

## Numpy Threading (for comparison)

In [26]:
%%time
# Prepare output
os.makedirs(output_dir, exist_ok=True)
logging.info('Writing outputs to ' + output_dir)

# Process input files with a worker pool
with mp.Pool(processes=n_workers) as pool:
    process_func = partial(process_event, output_dir=output_dir,
                           phi_range=(-np.pi, np.pi), **config['selection'])
    pool.map(process_func, file_prefixes)

../prepareTracks.py:213: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(p_zr[0])


         781 function calls in 58.327 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       35   58.206    1.663   58.206    1.663 {method 'acquire' of '_thread.lock' objects}
        1    0.088    0.088    0.088    0.088 {method 'acquire' of '_multiprocessing.SemLock' objects}
        3    0.019    0.006    0.019    0.006 {built-in method posix.waitpid}
        1    0.010    0.010    0.010    0.010 {built-in method posix.fork}
        3    0.001    0.000    0.001    0.000 {built-in method _thread.start_new_thread}
        1    0.000    0.000   58.327   58.327 <string>:2(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method posix.mkdir}
        4    0.000    0.000    0.000    0.000 synchronize.py:50(__init__)
        1    0.000    0.000   58.327   58.327 {built-in method builtins.exec}
        1    0.000    0.000    0.014    0.014 pool.py:153(__init__)
        1    0.000    0.000    0.010    0.010 popen_

## Numba Threading

In [38]:
%%time
# Prepare output
os.makedirs(output_dir, exist_ok=True)
logging.info('Writing outputs to ' + output_dir)

# Process input files with a worker pool
with mp.Pool(processes=n_workers) as pool:
    process_func = partial(process_event, output_dir=output_dir,
                           phi_range=(-np.pi, np.pi), **config['selection'])
    pool.map(process_func, file_prefixes)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
[1] During: typing of argument at <ipython-input-37-185a1b861a1a> (11)

File "<ipython-input-37-185a1b861a1a>", line 11:
def calc_eta(r, z):
    theta = np.arctan2(r, z)
    ^

This error may have been caused by the following argument(s):
- argument 0: cannot determine Numba type of <class 'pandas.core.series.Series'>
- argument 1: cannot determine Numba type of <class 'pandas.core.series.Series'>

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-c

## JIT Functions

In [2]:
@njit
def calc_dphi(phi1, phi2):
    """Computes phi2-phi1 given in range [-pi,pi]"""
    dphi = phi2 - phi1
    dphi[dphi > np.pi] -= 2*np.pi
    dphi[dphi < -np.pi] += 2*np.pi
    return dphi

@njit
def calc_eta(r, z):
    theta = np.arctan2(r, z)
    return -1. * np.log(np.tan(theta / 2.))

def select_segments(hits1, hits2, phi_slope_max, z0_max):
    """
    Construct a list of selected segments from the pairings
    between hits1 and hits2, filtered with the specified
    phi slope and z0 criteria.

    Returns: pd DataFrame of (index_1, index_2), corresponding to the
    DataFrame hit label-indices in hits1 and hits2, respectively.
    """
    # Start with all possible pairs of hits
    keys = ['evtid', 'r', 'phi', 'z']
    hit_pairs = hits1[keys].reset_index().merge(
        hits2[keys].reset_index(), on='evtid', suffixes=('_1', '_2'))
    # Compute line through the points
    dphi = calc_dphi(hit_pairs.phi_1, hit_pairs.phi_2)
    dz = hit_pairs.z_2 - hit_pairs.z_1
    dr = hit_pairs.r_2 - hit_pairs.r_1
    phi_slope = dphi / dr
    z0 = hit_pairs.z_1 - hit_pairs.r_1 * dz / dr
    # Filter segments according to criteria
    good_seg_mask = (phi_slope.abs() < phi_slope_max) & (z0.abs() < z0_max)
    return hit_pairs[['index_1', 'index_2']][good_seg_mask]

def construct_graph(hits, layer_pairs,
                    phi_slope_max, z0_max,
                    feature_names, feature_scale, T_feature_names, T_feature_scale, max_tracks=40):
    """Construct one graph (e.g. from one event)"""

    # Loop over layer pairs and construct segments
    layer_groups = hits.groupby('layer')
    segments = []
    for (layer1, layer2) in layer_pairs:
        # Find and join all hit pairs
        try:
            hits1 = layer_groups.get_group(layer1)
            hits2 = layer_groups.get_group(layer2)
        # If an event has no hits on a layer, we get a KeyError.
        # In that case we just skip to the next layer pair
        except KeyError as e:
            logging.info('skipping empty layer: %s' % e)
            continue
        # Construct the segments
        segments.append(select_segments(hits1, hits2, phi_slope_max, z0_max))
    # Combine segments from all layer pairs
    segments = pd.concat(segments)

    # Prepare the graph matrices
    n_hits = hits.shape[0]
    n_edges = segments.shape[0]
    unique_pids = pd.unique(hits['particle_id'])
    
    X = (hits[feature_names].values / feature_scale).astype(np.float32)
    Ri = np.zeros((n_hits, n_edges), dtype=np.uint8)
    Ro = np.zeros((n_hits, n_edges), dtype=np.uint8)
    y_edges = np.zeros(n_edges, dtype=np.float32)
        
    I = hits['hit_id']

    # We have the segments' hits given by dataframe label,
    # so we need to translate into positional indices.
    # Use a series to map hit label-index onto positional-index.
    hit_idx = pd.Series(np.arange(n_hits), index=hits.index)
    seg_start = hit_idx.loc[segments.index_1].values
    seg_end = hit_idx.loc[segments.index_2].values

    # Now we can fill the association matrices.
    # Note that Ri maps hits onto their incoming edges,
    # which are actually segment endings.
    Ri[seg_end, np.arange(n_edges)] = 1
    Ro[seg_start, np.arange(n_edges)] = 1
    # Fill the segment labels
    pid1 = hits.particle_id.loc[segments.index_1].values
    pid2 = hits.particle_id.loc[segments.index_2].values
    
    y_edges[:] = (pid1 == pid2)
    pids = pid1*y_edges
    
    hits = hits.assign(**{'{}'.format(k):v for k,v in zip(T_feature_names[:5], np.zeros(len(hits)))})
    
    # Calculate track parameters
    for pid in unique_pids:
        hits.loc[hits.particle_id == pid, T_feature_names[:5]] = get_track_parameters(hits[hits.particle_id == pid]['x'].to_numpy(), hits[hits.particle_id == pid]['y'].to_numpy(), hits[hits.particle_id == pid]['z'].to_numpy())
    
    y_params = (hits[T_feature_names].values / T_feature_scale).astype(np.float32)
    
#     print("Finished constructing graph")
    # Return a tuple of the results
    return Graph(X, Ri, Ro, y_edges, y_params, pids), I

def select_hits(hits, truth, particles, pt_min=0):
    # Barrel volume and layer ids
    vlids = [(8,2), (8,4), (8,6), (8,8),
             (13,2), (13,4), (13,6), (13,8),
             (17,2), (17,4)]
    
    # Assign volume IDs
    s = 0
    layer_dic = {}
    for v in vlids:
        if v[0] not in layer_dic: layer_dic[v[0]] = {}
        layer_dic[v[0]].update({v[1] : s})
        s += 1
    volume_mask = np.isin(hits['volume_id'], list(layer_dic.keys()))
    hits = hits[volume_mask]
    layer_list = [layer_dic[row['volume_id']][row['layer_id']] for row in hits]
    hits['layer_id'] = layer_list
    hits = hits_vlid
    
    # Apply momentum cut
    particles = particles[np.sqrt(particles.px**2 + particles.py**2) > pt_min]
    pt = np.sqrt(particles.px**2 + particles.py**2)
    rfn.append_fields(particles, 'pt', pt, usemask=False)
    
    # Combine particles with truth vector
    filler_particles = np.zeros((len(truth),), dtype=[('q', '<i4'), ('pt', '<f4')])
    for pi in np.unique(particles['particle_id']):
        filler_particles[truth['particle_id'] == pi] = particles[particles['particle_id'] == pi][['q', 'pt']]
    truth = truth[filler_particles['pt'] > 0]
    filler_particles = filler_particles[filler_particles['pt']>0]
    truth = rfn.rec_append_fields(truth, ['q', 'pt'], [filler_particles['q'], filler_particles['pt']])
    
    # Select the data columns we need
    hits = rfn.join_by('hit_id', rfn.drop_fields(hits, ['volume_id', 'module_id'], usemask=False), rfn.drop_fields(truth, ['weight'], usemask=False), usemask=False)
    r = np.sqrt(hits['y']**2 + hits['x']**2)
    phi = np.arctan2(hits['y'], hits['x'])
    hits = rfn.append_fields(hits, ['r', 'phi'], [r,phi], usemask=False)
    
    # Remove duplicate hits
    hits = hits.loc[
        hits.groupby(['particle_id', 'layer'], as_index=False).r.idxmin()
    ]
    
    # Remove tracks with less than 3 hits
    pid_counts = hits.groupby(['particle_id']).size().to_frame(name = 'pidcount').reset_index()
    pid_counts = pid_counts[pid_counts.pidcount > 2]    
    
    hits = hits.loc[hits['particle_id'].isin(pid_counts['particle_id'])]
    
    #This also removes tracks with less than 3 hits, but doesn't handle duplicates
#     hits = hits[hits['nhits'] > 2]
#     print("Finished selecting hits")
    return hits

def split_detector_sections(hits, phi_edges, eta_edges):
    """Split hits according to provided phi and eta boundaries."""
    hits_sections = []
    # Loop over sections
    for i in range(len(phi_edges) - 1):
        phi_min, phi_max = phi_edges[i], phi_edges[i+1]
        # Select hits in this phi section
        phi_hits = hits[(hits.phi > phi_min) & (hits.phi < phi_max)]
        # Center these hits on phi=0
        centered_phi = phi_hits.phi - (phi_min + phi_max) / 2
        phi_hits = phi_hits.assign(phi=centered_phi, phi_section=i)
        for j in range(len(eta_edges) - 1):
            eta_min, eta_max = eta_edges[j], eta_edges[j+1]
            # Select hits in this eta section
            eta = calc_eta(phi_hits.r, phi_hits.z)
            sec_hits = phi_hits[(eta > eta_min) & (eta < eta_max)]
            hits_sections.append(sec_hits.assign(eta_section=j))
    return hits_sections

@njit  
def quadratic_formula(a, b, c):
    if a == 0:
        return (-c/b, )
    x1 = (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)
    x2 = (-b - np.sqrt(b**2 - 4*a*c)) / (2*a)
    return (x1, x2)

@njit
def calc_R(xc, yc):
    return np.sqrt((x-xc)**2 + (y-yc)**2)

@njit
def fnc(c):
    Ri = calc_R(*c)
    return Ri - Ri.mean()

@vectorize
def get_track_parameters(x, y, z):
    # find the center of helix in x-y plane

    r3 = np.sqrt(x**2 + y**2 + z**2)
    p_zr0 = np.polyfit(r3, z, 1, full=True)
    res0 = p_zr0[1][0]/x.shape[0]
    p_zr = p_zr0[0]

    theta = np.arccos(p_zr[0])
    # theta = np.arccos(z[0]/r3[0])
    eta = -np.log(np.tan(theta/2.))

    center_estimate = np.mean(x), np.mean(y)
    trans_center, ier = optimize.leastsq(fnc, center_estimate)
    x0, y0 = trans_center
    R = calc_R(*trans_center).mean()

    # d0, z0
    d0 = abs(np.sqrt(x0**2 + y0**2) - R)

    r = np.sqrt(x**2 + y**2)
    p_rz = np.polyfit(r, z, 1)
    pp_rz = np.poly1d(p_rz)
    z0 = pp_rz(d0)

    # find the closest approaching point in x-y plane
    int_a = 1 + y0**2/x0**2
    int_b = -2*(x0 + y0**2/x0)
    int_c = x0**2 + y0**2 - R**2
    int_x0, int_x1 = quadratic_formula(int_a, int_b, int_c)
    x1 = int_x0 if abs(int_x0) < abs(int_x1) else int_x1
    y1 = y0*x1/x0
    phi = np.arctan2(y1, x1)

    # track travels colockwise or anti-colockwise
    # positive for colckwise
    xs = x[0] if x[0] != 0 else 1e-1
    ys = y[0] if y[0] != 0 else 1e-1
    is_14 = xs > 0
    is_above = y0 > ys/xs*x0
    sgn = 1 if is_14^is_above else -1

    # last entry is pT*(charge sign)
    return (d0, z0, phi, eta, 0.6*sgn*R/1000)

def process_event(prefix, output_dir, pt_min, n_eta_sections, n_phi_sections,
                  eta_range, phi_range, phi_slope_max, z0_max):
    # Load the data
    evtid = int(prefix[-9:])
    logging.info('Event %i, loading data' % evtid)
#     print('Event %i, loading data' % evtid)
    hits, particles, truth = trackml.dataset.load_event(
        prefix, parts=['hits', 'particles', 'truth'])

    # Convert Pandas to Numpy
    hits, particles, truth = hits.to_records(index=False), particles.to_records(index=False), truth.to_records(index=False)
    
    # Apply hit selection
    logging.info('Event %i, selecting hits' % evtid)
    hits = select_hits(hits, truth, particles, pt_min=pt_min).assign(evtid=evtid)

    # Divide detector into sections
    #phi_range = (-np.pi, np.pi)
    phi_edges = np.linspace(*phi_range, num=n_phi_sections+1)
    eta_edges = np.linspace(*eta_range, num=n_eta_sections+1)
    hits_sections = split_detector_sections(hits, phi_edges, eta_edges)

    # Geometric features and scale
    feature_names = 'r phi z'.split()
    d = dict((i,n) for n,i in enumerate(feature_names))
    feature_scale = np.array([1000., np.pi / n_phi_sections, 1000.])
    
    # Truth param features and scale
    T_feature_names = 'd0_T z0_T phi_T eta_T pt_T pt q tpx tpy tpz'.split()
    d = dict((i,n) for n,i in enumerate(col_names))
    T_feature_scale = np.array([100., 200., 5., 5., 20., 1., 1., 1., 1., 1.])
    
    # Define adjacent layers
    n_det_layers = 10
    l = np.arange(n_det_layers)
    layer_pairs = np.stack([l[:-1], l[1:]], axis=1)

    # Construct the graph
    logging.info('Event %i, constructing graphs' % evtid)
    tic = time.time()
    graphs_all = [construct_graph(section_hits, layer_pairs=layer_pairs,
                              phi_slope_max=phi_slope_max, z0_max=z0_max,
                              feature_names=feature_names,
                              feature_scale=feature_scale, 
                                T_feature_names=T_feature_names,
                                T_feature_scale=T_feature_scale)
              for section_hits in hits_sections]
    graphs = [x[0] for x in graphs_all]
    IDs    = [x[1] for x in graphs_all]
    # pids   = [x[2] for x in graphs_all]
    toc = time.time()
    logging.info('Graph construction time: %f' % (toc-tic))
    # Write these graphs to the output directory
    try:
        base_prefix = os.path.basename(prefix)
        filenames = [os.path.join(output_dir, '%s_g%03i' % (base_prefix, i))
                     for i in range(len(graphs))]
        filenames_ID = [os.path.join(output_dir, '%s_g%03i_ID' % (base_prefix, i))
                     for i in range(len(graphs))]
        filenames_pid = [os.path.join(output_dir, '%s_g%03i_pid' % (base_prefix, i))
                     for i in range(len(graphs))]
    except Exception as e:
        logging.info(e)
    logging.info('Event %i, writing graphs', evtid)
    save_graphs(graphs, filenames)
    for ID, file_name in zip(IDs, filenames_ID):
        np.savez(file_name, ID=ID)
    # for pid, file_name in zip(pids, filenames_pid):
    #     np.savez(file_name, pid=pid)



## JIT Play

In [2]:
import math
x = np.arange(100000000).reshape(10000, 10000)

def go_slow(a): # Function is compiled to machine code when called the first time
    trace = 0.
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
#     return a + trace              # Numba likes NumPy broadcasting

@cuda.jit # Set "nopython" mode for best performance, equivalent to @njit
def go_fast_gpu(a): # Function is compiled to machine code when called the first time
    trace = float(0.)
    for i in range(a.shape[0]):   # Numba likes loops
        trace += float(math.tanh(float(a[i, i]))) # Numba likes NumPy functions
#     a = a+trace              # Numba likes NumPy broadcasting

@njit # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = float(0.)
    for i in range(a.shape[0]):   # Numba likes loops
        trace += float(math.tanh(float(a[i, i]))) # Numba likes NumPy functions
#     a = a+trace              # Numba likes NumPy broadcasting

In [7]:
%%time
go_slow(x)

CPU times: user 15.4 ms, sys: 0 ns, total: 15.4 ms
Wall time: 15.2 ms


In [8]:
%%time
go_fast(x)

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 10.3 µs


## Pandas Functions

In [10]:
def calc_dphi(phi1, phi2):
    """Computes phi2-phi1 given in range [-pi,pi]"""
    dphi = phi2 - phi1
    dphi[dphi > np.pi] -= 2*np.pi
    dphi[dphi < -np.pi] += 2*np.pi
    return dphi

def calc_eta(r, z):
    theta = np.arctan2(r, z)
    return -1. * np.log(np.tan(theta / 2.))

def select_segments(hits1, hits2, phi_slope_max, z0_max):
    """
    Construct a list of selected segments from the pairings
    between hits1 and hits2, filtered with the specified
    phi slope and z0 criteria.

    Returns: pd DataFrame of (index_1, index_2), corresponding to the
    DataFrame hit label-indices in hits1 and hits2, respectively.
    """
    # Start with all possible pairs of hits
    keys = ['evtid', 'r', 'phi', 'z']
    hit_pairs = hits1[keys].reset_index().merge(
        hits2[keys].reset_index(), on='evtid', suffixes=('_1', '_2'))
    # Compute line through the points
    dphi = calc_dphi(hit_pairs.phi_1, hit_pairs.phi_2)
    dz = hit_pairs.z_2 - hit_pairs.z_1
    dr = hit_pairs.r_2 - hit_pairs.r_1
    phi_slope = dphi / dr
    z0 = hit_pairs.z_1 - hit_pairs.r_1 * dz / dr
    # Filter segments according to criteria
    good_seg_mask = (phi_slope.abs() < phi_slope_max) & (z0.abs() < z0_max)
    return hit_pairs[['index_1', 'index_2']][good_seg_mask]

def construct_graph(hits, layer_pairs,
                    phi_slope_max, z0_max,
                    feature_names, feature_scale, T_feature_names, T_feature_scale, max_tracks=40):
    """Construct one graph (e.g. from one event)"""

    # Loop over layer pairs and construct segments
    layer_groups = hits.groupby('layer')
    segments = []
    for (layer1, layer2) in layer_pairs:
        # Find and join all hit pairs
        try:
            hits1 = layer_groups.get_group(layer1)
            hits2 = layer_groups.get_group(layer2)
        # If an event has no hits on a layer, we get a KeyError.
        # In that case we just skip to the next layer pair
        except KeyError as e:
            logging.info('skipping empty layer: %s' % e)
            continue
        # Construct the segments
        segments.append(select_segments(hits1, hits2, phi_slope_max, z0_max))
    # Combine segments from all layer pairs
    segments = pd.concat(segments)

    # Prepare the graph matrices
    n_hits = hits.shape[0]
    n_edges = segments.shape[0]
    unique_pids = pd.unique(hits['particle_id'])
    
    X = (hits[feature_names].values / feature_scale).astype(np.float32)
    Ri = np.zeros((n_hits, n_edges), dtype=np.uint8)
    Ro = np.zeros((n_hits, n_edges), dtype=np.uint8)
    y_edges = np.zeros(n_edges, dtype=np.float32)
        
    I = hits['hit_id']

    # We have the segments' hits given by dataframe label,
    # so we need to translate into positional indices.
    # Use a series to map hit label-index onto positional-index.
    hit_idx = pd.Series(np.arange(n_hits), index=hits.index)
    seg_start = hit_idx.loc[segments.index_1].values
    seg_end = hit_idx.loc[segments.index_2].values

    # Now we can fill the association matrices.
    # Note that Ri maps hits onto their incoming edges,
    # which are actually segment endings.
    Ri[seg_end, np.arange(n_edges)] = 1
    Ro[seg_start, np.arange(n_edges)] = 1
    # Fill the segment labels
    pid1 = hits.particle_id.loc[segments.index_1].values
    pid2 = hits.particle_id.loc[segments.index_2].values
    
    y_edges[:] = (pid1 == pid2)
    pids = pid1*y_edges

    y_params = (hits[T_feature_names].values / T_feature_scale).astype(np.float32)
    
#     print("Finished constructing graph")
    # Return a tuple of the results
    return Graph(X, Ri, Ro, y_edges, y_params, pids), I

def select_hits(hits, truth, particles, pt_min=0):
    # Barrel volume and layer ids
    vlids = [(8,2), (8,4), (8,6), (8,8),
             (13,2), (13,4), (13,6), (13,8),
             (17,2), (17,4)]
    n_det_layers = len(vlids)
    # Select barrel layers and assign convenient layer number [0-9]
    vlid_groups = hits.groupby(['volume_id', 'layer_id'])
    hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer=i)
                      for i in range(n_det_layers)])
    # Calculate particle transverse momentum
    pt = np.sqrt(particles.px**2 + particles.py**2)
    # True particle selection.
    # Applies pt cut, removes all noise hits.
    particles = particles[pt > pt_min]
    particles = particles.join(pd.DataFrame(pt, columns=["pt"]))
    
    truth = (truth[['hit_id', 'particle_id', 'tpx', 'tpy', 'tpz']]
             .merge(particles[['particle_id', 'q', 'pt']], on='particle_id'))
    # Calculate derived hits variables
    r = np.sqrt(hits.x**2 + hits.y**2)
    phi = np.arctan2(hits.y, hits.x)
    # Select the data columns we need
    hits = (hits[['hit_id', 'x', 'y', 'z', 'layer']]
            .assign(r=r, phi=phi)
            .merge(truth[['hit_id', 'particle_id', 'pt', 'q', 'tpx', 'tpy', 'tpz']], on='hit_id'))
    # Remove duplicate hits
    hits = hits.loc[
        hits.groupby(['particle_id', 'layer'], as_index=False).r.idxmin()
    ]
    
    # Remove tracks with less than 3 hits
    pid_counts = hits.groupby(['particle_id']).size().to_frame(name = 'pidcount').reset_index()
    pid_counts = pid_counts[pid_counts.pidcount > 2]    
    
    hits = hits.loc[hits['particle_id'].isin(pid_counts['particle_id'])]
    
    #This also removes tracks with less than 3 hits, but doesn't handle duplicates
#     hits = hits[hits['nhits'] > 2]
#     print("Finished selecting hits")
    return hits

def split_detector_sections(hits, phi_edges, eta_edges):
    """Split hits according to provided phi and eta boundaries."""
    hits_sections = []
    # Loop over sections
    for i in range(len(phi_edges) - 1):
        phi_min, phi_max = phi_edges[i], phi_edges[i+1]
        # Select hits in this phi section
        phi_hits = hits[(hits.phi > phi_min) & (hits.phi < phi_max)]
        # Center these hits on phi=0
        centered_phi = phi_hits.phi - (phi_min + phi_max) / 2
        phi_hits = phi_hits.assign(phi=centered_phi, phi_section=i)
        for j in range(len(eta_edges) - 1):
            eta_min, eta_max = eta_edges[j], eta_edges[j+1]
            # Select hits in this eta section
            eta = calc_eta(phi_hits.r, phi_hits.z)
            sec_hits = phi_hits[(eta > eta_min) & (eta < eta_max)]
            hits_sections.append(sec_hits.assign(eta_section=j))
    return hits_sections

def get_track_parameters(x, y, z):
    # find the center of helix in x-y plane
    def calc_R(xc, yc):
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def fnc(c):
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    r3 = np.sqrt(x**2 + y**2 + z**2)
    p_zr0 = np.polyfit(r3, z, 1, full=True)
    res0 = p_zr0[1][0]/x.shape[0]
    p_zr = p_zr0[0]

    theta = np.arccos(p_zr[0])
    # theta = np.arccos(z[0]/r3[0])
    eta = -np.log(np.tan(theta/2.))

    center_estimate = np.mean(x), np.mean(y)
    trans_center, ier = optimize.leastsq(fnc, center_estimate)
    x0, y0 = trans_center
    R = calc_R(*trans_center).mean()

    # d0, z0
    d0 = abs(np.sqrt(x0**2 + y0**2) - R)

    r = np.sqrt(x**2 + y**2)
    p_rz = np.polyfit(r, z, 1)
    pp_rz = np.poly1d(p_rz)
    z0 = pp_rz(d0)


    def quadratic_formula(a, b, c):
        if a == 0:
            return (-c/b, )
        x1 = (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)
        x2 = (-b - np.sqrt(b**2 - 4*a*c)) / (2*a)
        return (x1, x2)

    # find the closest approaching point in x-y plane
    int_a = 1 + y0**2/x0**2
    int_b = -2*(x0 + y0**2/x0)
    int_c = x0**2 + y0**2 - R**2
    int_x0, int_x1 = quadratic_formula(int_a, int_b, int_c)
    x1 = int_x0 if abs(int_x0) < abs(int_x1) else int_x1
    y1 = y0*x1/x0
    phi = np.arctan2(y1, x1)

    # track travels colockwise or anti-colockwise
    # positive for colckwise
    xs = x[0] if x[0] != 0 else 1e-1
    ys = y[0] if y[0] != 0 else 1e-1
    is_14 = xs > 0
    is_above = y0 > ys/xs*x0
    sgn = 1 if is_14^is_above else -1

    # last entry is pT*(charge sign)
    return (d0, z0, phi, eta, 0.6*sgn*R/1000)

def process_event(prefix, output_dir, pt_min, n_eta_sections, n_phi_sections,
                  eta_range, phi_range, phi_slope_max, z0_max):
    # Load the data
    evtid = int(prefix[-9:])
    logging.info('Event %i, loading data' % evtid)
#     print('Event %i, loading data' % evtid)
    hits, particles, truth = trackml.dataset.load_event(
        prefix, parts=['hits', 'particles', 'truth'])

    # Apply hit selection
    logging.info('Event %i, selecting hits' % evtid)
    hits = select_hits(hits, truth, particles, pt_min=pt_min).assign(evtid=evtid)

    # Geometric features and scale
    feature_names = ['r', 'phi', 'z']
    feature_scale = np.array([1000., np.pi / n_phi_sections, 1000.])
    
    # Track param features and scale
    T_feature_names = ['d0_T', 'z0_T', 'phi_T', 'eta_T', 'pt_T', 'pt', 'q', 'tpx', 'tpy', 'tpz']
    T_feature_scale = np.array([100., 200., 5., 5., 20., 1., 1., 1., 1., 1.])

    # Set up columns for new hit parameters
    hits = hits.assign(**{'{}'.format(k):v for k,v in zip(T_feature_names[:5], np.zeros(len(hits)))})
    
    unique_pids = pd.unique(hits['particle_id'])
    # Calculate track parameters
    for pid in unique_pids:
        hits.loc[hits.particle_id == pid, T_feature_names[:5]] = get_track_parameters(hits[hits.particle_id == pid]['x'].to_numpy(), hits[hits.particle_id == pid]['y'].to_numpy(), hits[hits.particle_id == pid]['z'].to_numpy())
    
    
    # Divide detector into sections
    #phi_range = (-np.pi, np.pi)
    phi_edges = np.linspace(*phi_range, num=n_phi_sections+1)
    eta_edges = np.linspace(*eta_range, num=n_eta_sections+1)
    hits_sections = split_detector_sections(hits, phi_edges, eta_edges)

    
    # Define adjacent layers
    n_det_layers = 10
    l = np.arange(n_det_layers)
    layer_pairs = np.stack([l[:-1], l[1:]], axis=1)

    # Construct the graph
    logging.info('Event %i, constructing graphs' % evtid)
    tic = time.time()
    graphs_all = [construct_graph(section_hits, layer_pairs=layer_pairs,
                              phi_slope_max=phi_slope_max, z0_max=z0_max,
                              feature_names=feature_names,
                              feature_scale=feature_scale, 
                                T_feature_names=T_feature_names,
                                T_feature_scale=T_feature_scale)
              for section_hits in hits_sections]
    graphs = [x[0] for x in graphs_all]
    IDs    = [x[1] for x in graphs_all]
    # pids   = [x[2] for x in graphs_all]
    toc = time.time()
    logging.info('Graph construction time: %f' % (toc-tic))
    # Write these graphs to the output directory
    try:
        base_prefix = os.path.basename(prefix)
        filenames = [os.path.join(output_dir, '%s_g%03i' % (base_prefix, i))
                     for i in range(len(graphs))]
        filenames_ID = [os.path.join(output_dir, '%s_g%03i_ID' % (base_prefix, i))
                     for i in range(len(graphs))]
        filenames_pid = [os.path.join(output_dir, '%s_g%03i_pid' % (base_prefix, i))
                     for i in range(len(graphs))]
    except Exception as e:
        logging.info(e)
    logging.info('Event %i, writing graphs', evtid)
    save_graphs(graphs, filenames)
    for ID, file_name in zip(IDs, filenames_ID):
        np.savez(file_name, ID=ID)
    # for pid, file_name in zip(pids, filenames_pid):
    #     np.savez(file_name, pid=pid)


# Testing

In [18]:
pt_min, n_phi_sections, phi_slope_max, z0_max = 1, 1, 0.001, 150
event_prefix = 'event000001000'
hits, particles, truth = load_event(os.path.join('/global/cscratch1/sd/danieltm/ExaTrkX/trackml/train_100_events/', event_prefix), 
                                    parts=['hits', 'particles', 'truth'])
evtid = int(event_prefix[-9:])
hits = hits.assign(evtid=evtid)

## Numba Version

Set up the inital constraints and choose an event

In [118]:
hits_np = rfn.append_fields(hits_vlid, 'layer_num', layer_list, usemask=False)

In [235]:
truth.dtype.descr + [('q', '<i4'), ('pt', '<f4')]

[('hit_id', '<i4'),
 ('particle_id', '<i8'),
 ('tx', '<f4'),
 ('ty', '<f4'),
 ('tz', '<f4'),
 ('tpx', '<f4'),
 ('tpy', '<f4'),
 ('tpz', '<f4'),
 ('weight', '<f4'),
 ('q', '<i4'),
 ('pt', '<f4')]

Select end-cap volumes

In [19]:
# This is ONLY the end-cap
# vlids = [(9,2), (9,4), (9,6), (9,8), (9,10), (9,12), (9,14)]
# These are the other front detectors
#                     (14,2), (14,4), (14,6), (14,8), (14,10), (14,12),
#                     (18,2), (18,4), (18,6), (18,8), (18,10), (18,12)]
# These are the barrel volumes
vlids = [(8,2), (8,4), (8,6), (8,8),
             (13,2), (13,4), (13,6), (13,8),
             (17,2), (17,4)]
# eta_region = [0,0.2]

# n_det_layers = len(vlids)
# vlid_groups = hits.groupby(['volume_id', 'layer_id'])
# hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer=i)
#                       for i in range(n_det_layers)])

In [20]:
# Converts to structured array
hits, particles, truth = hits.to_records(index=False), particles.to_records(index=False), truth.to_records(index=False)
s = 0
layer_dic = {}

# Makes unique layer labels
for v in vlids:
    if v[0] not in layer_dic: layer_dic[v[0]] = {}
    layer_dic[v[0]].update({v[1] : s})
    s += 1
volume_mask = np.isin(hits['volume_id'], list(layer_dic.keys()))
hits = hits[volume_mask]
layer_list = [layer_dic[row['volume_id']][row['layer_id']] for row in hits]
hits['layer_id'] = layer_list

In [7]:
hits

rec.array([( 16874,   -32.5544,  -3.64871, -469.865,  8, 0,    1, 1000),
           ( 16875,   -33.1537,  -1.93474, -423.517,  8, 0,    1, 1000),
           ( 16876,   -26.3624, -18.4237 , -461.375,  8, 0,    2, 1000),
           ...,
           (118006, -1021.59  ,  53.764  , 1035.4  , 17, 9, 3191, 1000),
           (118007, -1022.28  ,  15.3443 ,  981.4  , 17, 9, 3192, 1000),
           (118008, -1022.57  ,  13.6296 , 1057.   , 17, 9, 3192, 1000)],
          dtype=[('hit_id', '<i4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('volume_id', '<i4'), ('layer_id', '<i4'), ('module_id', '<i4'), ('evtid', '<i8')])

In [21]:
particles = particles[np.sqrt(particles.px**2 + particles.py**2) > pt_min]
pt = np.sqrt(particles.px**2 + particles.py**2)
particles = rfn.append_fields(particles, 'pt', pt, usemask=False)

In [22]:
filler_particles = np.zeros((len(truth),), dtype=[('q', '<i4'), ('pt', '<f4')])
for pi in np.unique(particles['particle_id']):
    filler_particles[truth['particle_id'] == pi] = particles[particles['particle_id'] == pi][['q', 'pt']]
truth = truth[filler_particles['pt'] > 0]
filler_particles = filler_particles[filler_particles['pt']>0]
truth = rfn.rec_append_fields(truth, ['q', 'pt'], [filler_particles['q'], filler_particles['pt']])

In [23]:
theta_hits = np.arccos(hits.z / (hits.x**2 + hits.y**2 + hits.z**2)**(0.5))
eta_hits = -np.log(np.tan(theta_hits/2))
phi_hits = np.arctan2(hits.y, hits.x)
r_hits = np.sqrt(hits.x**2 + hits.y**2)

In [11]:
particles.shape

(1383,)

In [12]:
hits

rec.array([( 16874,   -32.5544,  -3.64871, -469.865,  8, 0,    1, 1000),
           ( 16875,   -33.1537,  -1.93474, -423.517,  8, 0,    1, 1000),
           ( 16876,   -26.3624, -18.4237 , -461.375,  8, 0,    2, 1000),
           ...,
           (118006, -1021.59  ,  53.764  , 1035.4  , 17, 9, 3191, 1000),
           (118007, -1022.28  ,  15.3443 ,  981.4  , 17, 9, 3192, 1000),
           (118008, -1022.57  ,  13.6296 , 1057.   , 17, 9, 3192, 1000)],
          dtype=[('hit_id', '<i4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('volume_id', '<i4'), ('layer_id', '<i4'), ('module_id', '<i4'), ('evtid', '<i8')])

In [59]:
@njit
def mask_test(hits):
    hits[hits['layer_id']==0]['particle_id'] = 7
    return hits[hits['layer_id']==0]['particle_id']

In [60]:
%%time
mask_test(hits)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function setitem>) with argument(s) of type(s): (unaligned array(Record(hit_id[type=int32;offset=0],x[type=float32;offset=4],y[type=float32;offset=8],z[type=float32;offset=12],layer_id[type=int32;offset=16],evtid[type=int64;offset=20],particle_id[type=int64;offset=28],tx[type=float32;offset=36],ty[type=float32;offset=40],tz[type=float32;offset=44],tpx[type=float32;offset=48],tpy[type=float32;offset=52],tpz[type=float32;offset=56],q[type=int32;offset=60],pt[type=float32;offset=64];68;False), 1d, C), Literal[str](particle_id), Literal[int](7))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    All templates rejected with literals.
In definition 5:
    All tem

In [24]:
hits = rfn.join_by('hit_id', rfn.drop_fields(hits, ['volume_id', 'module_id'], usemask=False), rfn.drop_fields(truth, ['weight'], usemask=False), usemask=False)

In [25]:
r = np.sqrt(hits['y']**2 + hits['x']**2)
phi = np.arctan2(hits['y'], hits['x'])
hits = rfn.append_fields(hits, ['r', 'phi'], [r,phi], usemask=False)

In [85]:
# dt = np.dtype([('dup', int64)])
@njit
def rem_dup(hits):
    dup_hits = np.zeros((len(hits),), dtype=boolean)
    # Drop duplicates
    layers = np.unique(hits['layer_id'])
#     print(layers)
    for li in prange(len(layers)):
        matching = np.unique(hits[hits['layer_id'] == layers[li]]['particle_id'])
        for pi in prange(len(matching)):
            unique_hit = np.where((hits['particle_id']==matching[pi]) & (hits['layer_id']==layers[li]) & (np.min(hits[(hits['particle_id']==matching[pi]) & (hits['layer_id']==layers[li])]['r']) == hits['r']))
            dup_hits[unique_hit] = True 
#         pass
    return dup_hits

In [141]:
%%time
dup_hits = np.zeros((len(hits),), dtype=[('dup', 'bool')])
# Drop duplicates
for li in np.unique(hits['layer_id']):
    for pi in np.unique(hits[hits['layer_id'] == li]['particle_id']):
        dup_hits[(hits['particle_id']==pi) & (hits['layer_id']==li) & (np.min(hits[(hits['particle_id']==pi) & (hits['layer_id']==li)]['r']) == hits['r'])] = True
new_hits = hits[dup_hits['dup']]

CPU times: user 2.37 s, sys: 1.25 ms, total: 2.37 s
Wall time: 2.48 s


In [87]:
%%time

new_hits = rem_dup(hits)
# rem_dup.parallel_diagnostics(level=2)

CPU times: user 434 ms, sys: 0 ns, total: 434 ms
Wall time: 432 ms


In [125]:
sum(new_hits['dup'])

6437

In [56]:
hits[(hits['particle_id']==517919111108362240) & (hits['layer_id']==0)]['r']

array([32.077705, 31.579145], dtype=float32)

In [63]:
np.argmin(hits[(hits['particle_id']==517919111108362240) & (hits['layer_id']==0)]['r'])

1

In [77]:
dup_hits[(hits['particle_id']==517919111108362240) & (hits['layer_id']==0)]

array([(False,), ( True,)], dtype=[('dup', '?')])

In [73]:
dup_hits[(hits['particle_id']==517919111108362240) & (hits['layer_id']==0) & (np.min(hits[(hits['particle_id']==517919111108362240) & (hits['layer_id']==0)]['r']) == hits['r'])] = True

In [66]:
np.min(hits[(hits['particle_id']==517919111108362240) & (hits['layer_id']==0)]['r']) == hits[(hits['particle_id']==517919111108362240) & (hits['layer_id']==0)]['r']

array([False,  True])

In [32]:
# Create big hits + truth D.F.
hits = (hits[['hit_id', 'x', 'y', 'z', 'layer', 'evtid']]
            .assign(r=r_hits, phi=phi_hits, eta=eta_hits)
            .merge(truth[['hit_id', 'particle_id', 'nhits', 'pt']], on='hit_id'))
# Remove duplicated
hits = hits.loc[
        hits.groupby(['particle_id', 'layer'], as_index=False).r.idxmin()
    ]
hits = hits[hits['nhits'] > 2]

In [89]:
hits = hits.assign(**{'{}'.format(k):v for k,v in zip(["C", "D","E","F","G"], np.zeros(len(hits)))})

In [102]:
for pid in pd.unique(hits['particle_id']):
    hits.loc[hits.particle_id == pid, ["C", "D", "E", "F", "G"]] = get_track_parameters(hits[hits.particle_id == pid]['x'].to_numpy(), hits[hits.particle_id == pid]['y'].to_numpy(), hits[hits.particle_id == pid]['z'].to_numpy())

Select certain eta region

In [9]:
eta_region = [3.2,4]
hits = hits[(hits.eta > eta_region[0]) & (hits.eta < eta_region[1])]
# hits = hits.sort_values(by='eta')

Organise into adjacent layers

In [10]:
l = np.arange(n_det_layers)
layer_pairs = np.stack([l[:-1], l[1:]], axis=1)

In [11]:
feature_names = ['r', 'phi', 'z']
feature_scale = np.array([1000., np.pi / n_phi_sections, 1000.])

In [12]:
graph, IDs = construct_graph(hits, layer_pairs=layer_pairs,
                              phi_slope_max=phi_slope_max, z0_max=z0_max,
                              feature_names=feature_names,
                              feature_scale=feature_scale)

In [13]:
Ri_rows, Ri_cols = graph.Ri.nonzero()
Ro_rows, Ro_cols = graph.Ro.nonzero()
n_edges = Ri_cols.shape[0]
edge_index = np.zeros((2, n_edges), dtype=int)
edge_index[0, Ro_cols] = Ro_rows
edge_index[1, Ri_cols] = Ri_rows
print(len(graph.X), " hits and ", edge_index.shape[1], " edges")

634  hits and  524  edges


In [14]:
X, Y, edges, labels = graph.X, graph.y, edge_index, graph.y
X = X*feature_scale

## Pandas Version

In [20]:
pt_min, n_phi_sections, phi_slope_max, z0_max = 1, 1, 0.001, 150
event_prefix = 'event000001000'
hits, particles, truth = load_event(os.path.join('/global/cscratch1/sd/danieltm/ExaTrkX/trackml/train_100_events/', event_prefix), 
                                    parts=['hits', 'particles', 'truth'])
evtid = int(event_prefix[-9:])
hits = hits.assign(evtid=evtid)

# This is ONLY the end-cap
# vlids = [(9,2), (9,4), (9,6), (9,8), (9,10), (9,12), (9,14)]
# These are the other front detectors
#                     (14,2), (14,4), (14,6), (14,8), (14,10), (14,12),
#                     (18,2), (18,4), (18,6), (18,8), (18,10), (18,12)]
# These are the barrel volumes
vlids = [(8,2), (8,4), (8,6), (8,8),
             (13,2), (13,4), (13,6), (13,8),
             (17,2), (17,4)]
# eta_region = [0,0.2]

n_det_layers = len(vlids)
vlid_groups = hits.groupby(['volume_id', 'layer_id'])
hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer=i)
                      for i in range(n_det_layers)])

In [50]:
pt = np.sqrt(particles.px**2 + particles.py**2)
particles = particles[pt > pt_min]
particles = particles.join(pd.DataFrame(pt, columns=["pt"]))

In [51]:
truth = (truth[['hit_id', 'particle_id', 'tpx', 'tpy', 'tpz']]
             .merge(particles[['particle_id', 'q', 'pt']], on='particle_id'))

In [52]:
theta_hits = np.arccos(hits.z / (hits.x**2 + hits.y**2 + hits.z**2)**(0.5))
eta_hits = -np.log(np.tan(theta_hits/2))
phi_hits = np.arctan2(hits.y, hits.x)
r_hits = np.sqrt(hits.x**2 + hits.y**2)

In [53]:
# Create big hits + truth D.F.
hits = (hits[['hit_id', 'x', 'y', 'z', 'layer', 'evtid']]
        .assign(r=r_hits, phi=phi_hits, eta=eta_hits)
        .merge(truth[['hit_id', 'particle_id', 'pt', 'q', 'tpx', 'tpy', 'tpz']], on='hit_id'))
# Remove duplicated
hits = hits.loc[
        hits.groupby(['particle_id', 'layer'], as_index=False).r.idxmin()
    ]

# Remove tracks with less than 3 hits
pid_counts = hits.groupby(['particle_id']).size().to_frame(name = 'pidcount').reset_index()
pid_counts = pid_counts[pid_counts.pidcount > 2]    

hits = hits.loc[hits['particle_id'].isin(pid_counts['particle_id'])]
# hits = hits[hits['nhits'] > 2]

In [54]:
# Track param features and scale
T_feature_names = ['d0_T', 'z0_T', 'phi_T', 'eta_T', 'pt_T', 'pt', 'q', 'tpx', 'tpy', 'tpz']
T_feature_scale = np.array([100., 200., 5., 5., 20., 1., 1., 1., 1., 1.])

# Set up columns for new hit parameters
hits = hits.assign(**{'{}'.format(k):v for k,v in zip(T_feature_names[:5], np.zeros(len(hits)))})
unique_pids = pd.unique(hits['particle_id'])
# Calculate track parameters

for pid in unique_pids:
    hits.loc[hits.particle_id == pid, T_feature_names[:5]] = get_track_parameters(hits[hits.particle_id == pid]['x'].to_numpy(), hits[hits.particle_id == pid]['y'].to_numpy(), hits[hits.particle_id == pid]['z'].to_numpy())

Select certain eta region

In [44]:
eta_region = [-5,5]
hits = hits[(hits.eta > eta_region[0]) & (hits.eta < eta_region[1])]
# hits = hits.sort_values(by='eta')

Organise into adjacent layers

In [45]:
l = np.arange(n_det_layers)
layer_pairs = np.stack([l[:-1], l[1:]], axis=1)

In [46]:
feature_names = ['r', 'phi', 'z']
feature_scale = np.array([1000., np.pi / n_phi_sections, 1000.])

In [48]:
graph, IDs = construct_graph(hits, layer_pairs=layer_pairs,
                              phi_slope_max=phi_slope_max, z0_max=z0_max,
                              feature_names=feature_names,
                              feature_scale=feature_scale, T_feature_names = T_feature_names, T_feature_scale = T_feature_scale)